In [ ]:
#default_exp pleiades

# Pleiades

> Working through conversion to cogs, upload to object storage, stac item metadata parsing, etc. for pleiades datasets

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
from glob import glob
import time
import numpy as np
from datetime import datetime
import json

import xmltodict
import pystac
from pystac import STAC_IO
from pystac.extensions.eo import Band
import geopandas as gpd

from sac_stac.utils import sedas_client, sedas_find_datasets, sedas_download, sedas_extract
from sac_stac.utils import cogmosaicbands
from sac_stac.utils import s3_upload_dir, s3_list_objects_paths, clean_up
from sac_stac.utils import pystac_setIO, create_uri

In [ ]:
import pandas as pd

## **Preparation**: Download, cloud-optimise, upload

### Prep Function

Download and use basic gdal tools to mosaic any tiles into single images, convert to cog and upload to our object storage.

In [ ]:
#export
def prep_pleiades(sedas_supplierId, inter_dir="/tmp/data/intermediate/", 
                  s3_bucket="public-eo-data", s3_dir="uksa-ssgp/pleiades/"):
    try:
        inter_dir = f"{inter_dir}{sedas_supplierId}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        scene_name = sedas_supplierId
        down_zip = f"{inter_dir}{scene_name}.zip"
        scene_dir = f"{down_zip[:-4]}/"
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {scene_name} within {inter_dir}")
        # find & download
        sedas_scene_res = sedas_client().search_product(sedas_supplierId)[0]
        sedas_download([sedas_scene_res], inter_dir)
        sedas_extract(down_zip, scene_dir)
        # sensor-specific band mosaicing and cogifying
        imgs_ms = glob(f"{scene_dir}*/*MS_002*/*.TIF")
        imgs_pan = glob(f"{scene_dir}*/*P_001*/*.TIF")
        cogmosaicbands(imgs_pan, 1, imgs_pan[0][:-13])
        cogmosaicbands(imgs_ms, 4, imgs_ms[0][:-13])
        # upload
        s3_upload_dir(scene_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {scene_name} at {s3_dir}{scene_name}/")
        clean_up(inter_dir)
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
        clean_up(inter_dir)

### Iteration with samples

Some samples used to iterate creation of the prep function.

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "Pleiades"
                            )
pd.DataFrame(result['products']).head(2)

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,c0ec9a5f12356e87bc910ddbc49dbb76,Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.654728 51.309517,-1.345414 51.3081...",2018-10-24T11:17:22Z,...,4.825125e+08,1.0,5.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'cf...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,94cc7887414be0912de7ca44288f79da,Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-2.153852 51.603313,-1.841533 51.6033...",2018-09-29T11:10:08Z,...,1.565260e+08,2.0,22.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...


Example of testing the first one.

In [ ]:
done = ["Pleiades_UKSA7_SO18034613-7-01_DS_PHR1A_201802241127550_FR1_PX_W002N51_0711_024843613631101",
        "Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101",
        "Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_201807241124303_FR1_PX_W002N51_0720_01561TPP1601069279",
        "Pleiades_UKSA341_SO18034616-41-01_DS_PHR1B_201809021117346_FR1_PX_W002N51_1008_03279TPP1601069492",
        "Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318",
        "Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712"]

In [ ]:
#hide
# for p in pd.DataFrame(result['products']).supplierId.values:
    
#     if not True in [i.split('_')[1] in p for i in done]:
#         prep_pleiades(p)

In [ ]:
# prep_pleiades("Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712")

### Job lists

Once happy with `prep_pleiades` we create a list of jobs from search results of `sedas_find_datasets` to submit to a redis queue as per `rediswq`.

We can test these via redis within `pleiades_prep_worker`.

Sometimes we run as campaigns via K8s.

In [ ]:
jobs_dir = "/tmp/data/"
td = datetime.today()
td = td.strftime('%Y')+td.strftime('%m')+td.strftime('%d')
td

'20201111'

In [ ]:
with open(f"{jobs_dir}/JOBLIST_{td}_pleiades.txt", 'w') as t:
    for v in pd.DataFrame(result['products']).supplierId.values:
        t.write("rpush jobPL"+" '{"+'"sedas_supplierId": "'+ v +'"'"}'" + '\n')

## Parse core (& limited extension) STAC metadata

With some cogs hosted on object storage we can fill out some of the necessary STAC core and extension metadata fields.

### *Collection* fields

Since for Pleiades we're working with electro-optical imagery, we can make use of the STAC **[EO Extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/eo)**. At the Pleiades *Collection* level we can therefore manually set some default metadata fields that will be consistent across all *Items* and *Assets*.

In [ ]:
#export
pleiades_bands = [Band.create(name='Panchromatic', description='Panchromatic: 480 - 830 nm', common_name='pan'),
                  Band.create(name='Blue', description='Blue: 430 - 550 nm', common_name='blue'),
                  Band.create(name='Green', description='Green: 490 - 610 nm', common_name='green'),
                  Band.create(name='Red', description='Red: 600 - 720 nm', common_name='red'),
                  Band.create(name='Near-Infrared', description='Near-Infrared: 750 - 950 nm', common_name='nir')]

We could also make use of the **[Satellite Extension](https://github.com/radiantearth/stac-spec/blob/master/extensions/sat/README.md)**, however the fields of orbit_state (i.e. ascending / descending) and relative_orbit (i.e. orbit no.) don't seem immediately useful for Pleiades. Other extensions are commented on as we delve deeper into a *Collection*.

### *Item* fields

In [ ]:
obj_paths_list = s3_list_objects_paths('public-eo-data', 'uksa-ssgp/pleiades/')

In [ ]:
# get unique Item / scene names (third dir from path)
scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
scene_names[:5]

['Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101',
 'Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201809291110083_FR1_PX_W002N51_0116_00596TPP1601069431',
 'Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_201802251121249_FR1_PX_W002N51_0314_01048',
 'Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_201802251121249_FR1_PX_W002N51_0314_010483613775101',
 'Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_201807241124303_FR1_PX_W002N51_0720_01561TPP1601069279']

In [ ]:
# can just work with one
scene_name = scene_names[0]
scene_name

'Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101'

In [ ]:
# and objects associated with that scene
scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
scene_obj_paths[:2]

['uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/DELIVERY.PDF',
 'uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_MS_002/DIM_PHR1A_MS_201809291110008_ORT_3614104101-2.XML']

In [ ]:
# set our own I/O for pystac
pystac_setIO()

We have a set of pleiades-specific tools for parsing stac metadata at the *Item* level. Some are more complex than others both in approach for generating and assumptions.

We start off simple by getting the date from the scene name.

In [ ]:
#export
def pleiades_get_dt(scene_name):
    return datetime.strptime(scene_name.split('_')[5][:14], '%Y%m%d%H%M%S')

In [ ]:
pleiades_get_dt(scene_name)

datetime.datetime(2018, 9, 29, 11, 10)

Geometry (footprint of actually useful data) can be taken from a .gml.

In [ ]:
#export
def pleiades_get_geom(scene_paths):
    roi_path = [i for i in scene_paths if (i.endswith('1_MSK.GML') * os.path.basename(i).startswith('ROI'))][0]
    roi_uri = create_uri(roi_path)
    return json.loads(gpd.read_file(roi_uri).to_crs('EPSG:4326').to_json(show_bbox=True))['features'][0]['geometry']

In [ ]:
pleiades_get_geom(scene_obj_paths)

{'type': 'Polygon',
 'coordinates': [[[-1.7162111146081491, 51.703447752207374],
   [-1.6539393674716631, 51.70305171212088],
   [-1.5922000556276548, 51.7022918404328],
   [-1.592478005824943, 51.66971308516982],
   [-1.5923937686249074, 51.660564184797174],
   [-1.5924483796708082, 51.65689143996495],
   [-1.5923435803877832, 51.6484257860413],
   [-1.5925225669770522, 51.629252456312805],
   [-1.6313240498998447, 51.629686309100045],
   [-1.6617769179346535, 51.62994873863878],
   [-1.715447257136181, 51.63018534903567],
   [-1.7483946591920845, 51.630161588463274],
   [-1.7789150489058498, 51.63009707066823],
   [-1.8369443638514822, 51.62966137158252],
   [-1.8706869752457782, 51.629244747190995],
   [-1.902984365251201, 51.62878127998801],
   [-1.9031756192477154, 51.64416997236793],
   [-1.9032281966762568, 51.65003679955556],
   [-1.9031830688830722, 51.650868451669226],
   [-1.9032398631646559, 51.65134952934474],
   [-1.9032635080951406, 51.6606374853309],
   [-1.903368845801

A variety of *Item* metadata already exists within either the multi-spectral and / or the panchromatic image metadata supplied with the original dataset.

In [ ]:
#export
def pleiades_parsemeta(meta_path):
    return xmltodict.parse(pystac.STAC_IO.read_text(create_uri(meta_path)))

In [ ]:
meta = pleiades_parsemeta([i for i in scene_obj_paths if (i.endswith('.XML')) & (os.path.basename(i).startswith('DIM'))][0]) # not sure if this catches for all Items?

Bounding box is one..

In [ ]:
#export
def pleiades_get_bbox(metadata):
    lons = [float(i['LON']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    lats = [float(i['LAT']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    return [min(lons), min(lats), max(lons), max(lats)]

In [ ]:
pleiades_get_bbox(meta)

[-1.903199885312827, 51.62869184186537, -1.592242985175717, 51.70382196570706]

That's it to meet the **core specification** of STAC.

In [ ]:
item = pystac.Item(id=scene_name,
                  datetime=pleiades_get_dt(scene_name),
                  geometry=pleiades_get_geom(scene_obj_paths),
                  bbox=pleiades_get_bbox(meta),
                  properties={})

In [ ]:
item.validate()#, item.to_dict()

We can now focus on adding useful extensions.

In [ ]:
def pleiades_get_gsd(metadata):
    across = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ACROSS_TRACK']['#text'])
    along = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ALONG_TRACK']['#text'])
    return round(( across + along ) / 2, 2)

In [ ]:
pleiades_get_gsd(meta)

0.74

In [ ]:
item.common_metadata.gsd = pleiades_get_gsd(meta)

In [ ]:
item.ext.enable('eo')

In [ ]:
def pleiades_get_cloudcover(metadata):
    return round(float(metadata['Dimap_Document']['Dataset_Content']['CLOUD_COVERAGE']['#text']),2)

In [ ]:
pleiades_get_cloudcover(meta)

0.0

In [ ]:
item.ext.eo.cloud_cover = pleiades_get_cloudcover(meta)

In [ ]:
item.ext.enable('projection')

In [ ]:
def pleiades_get_crs(metadata):
    return int(metadata['Dimap_Document']['Coordinate_Reference_System']['Projected_CRS']['PROJECTED_CRS_NAME'])

In [ ]:
pleiades_get_crs(meta)

27700

In [ ]:
item.ext.projection.epsg = pleiades_get_crs(meta)

In [ ]:
item.validate()

### Adding *Assets*

In [ ]:
#export
pleiades_bands = [Band.create(name='Panchromatic', description='Panchromatic: 480 - 830 nm', common_name='pan'),
                  Band.create(name='Blue', description='Blue: 430 - 550 nm', common_name='blue'),
                  Band.create(name='Green', description='Green: 490 - 610 nm', common_name='green'),
                  Band.create(name='Red', description='Red: 600 - 720 nm', common_name='red'),
                  Band.create(name='Near-Infrared', description='Near-Infrared: 750 - 950 nm', common_name='nir')]

In [ ]:
bfs = {
    'Panchromatic':{'ends':'_band1', 'dif':'_P_', 'id':'B0'},
    'Blue':{'ends':'_band1', 'dif':'_MS_', 'id':'B1'},
    'Green':{'ends':'_band2', 'dif':'_MS_', 'id':'B2'},
    'Red':{'ends':'_band3', 'dif':'_MS_', 'id':'B3'},
    'Near-Infrared':{'ends':'_band4', 'dif':'_MS_', 'id':'B4'}
}
bfs

{'Panchromatic': {'ends': '_band1', 'dif': '_P_', 'id': 'B0'},
 'Blue': {'ends': '_band1', 'dif': '_MS_', 'id': 'B1'},
 'Green': {'ends': '_band2', 'dif': '_MS_', 'id': 'B2'},
 'Red': {'ends': '_band3', 'dif': '_MS_', 'id': 'B3'},
 'Near-Infrared': {'ends': '_band4', 'dif': '_MS_', 'id': 'B4'}}

In [ ]:
for band in pleiades_bands:
    print(band.name)
    
    matched_paths = [ o for o in scene_obj_paths if (o.endswith(f"{bfs[band.name]['ends']}.tif")) & (f"{bfs[band.name]['dif']}" in o) ]
    if len(matched_paths) > 1:
        raise Exception(f"Found too many matches: {matched_paths}")
    band_url = create_uri(matched_paths[0])
    print(band_url)
    
    asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
    item.ext.eo.set_bands([band], asset)
    item.add_asset(bfs[band.name]['id'], asset)    

Panchromatic
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_P_001/IMG_PHR1A_P_201809291110008_ORT_36141041_band1.tif
Blue
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_MS_002/IMG_PHR1A_MS_201809291110008_ORT_36141041_band1.tif
Green
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_MS_002/IMG_PHR1A_MS_201809291110008_ORT_36141041_band2.tif
Red
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/IMG_PHR1A_MS_002/IMG_PHR1A_MS_201809291110008_ORT_36141041_band3.tif
Near-Infrared
http://s3-uk-1.sa-catapult.co.uk/pub

In [ ]:
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101',
 'properties': {'datetime': '2018-09-29T11:10:00Z',
  'gsd': 0.74,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-1.7162111146081491, 51.703447752207374],
    [-1.6539393674716631, 51.70305171212088],
    [-1.5922000556276548, 51.7022918404328],
    [-1.592478005824943, 51.66971308516982],
    [-1.5923937686249074, 51.660564184797174],
    [-1.5924483796708082, 51.65689143996495],
    [-1.5923435803877832, 51.6484257860413],
    [-1.5925225669770522, 51.629252456312805],
    [-1.6313240498998447, 51.629686309100045],
    [-1.6617769179346535, 51.62994873863878],
    [-1.715447257136181, 51.63018534903567],
    [-1.7483946591920845, 51.630161588463274],
    [-1.7789150489058498, 51.63009707066823],
    [-1.8369443638514822, 51.62966137158252],
    [-1.8706869752457782, 51.62924

### Building the *Collection*

In [ ]:
collection_id = 'uksa-ssgp-pleiades'
collection_id

'uksa-ssgp-pleiades'

In [ ]:
collection_title = 'SSGP-procured Pleiades images over the UK'
collection_title

'SSGP-procured Pleiades images over the UK'

In [ ]:
collection_description = '''### UKSA / SSGP Pleiades

A collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).
'''
print(collection_description)

### UKSA / SSGP Pleiades

A collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).



In [ ]:
spatial_extent = pystac.SpatialExtent([[-7.57216793459, 49.959999905, 1.68153079591, 58.6350001085]])
temporal_extent = pystac.TemporalExtent([[datetime(2011, 12, 16), None]])
collection_extent = pystac.Extent(spatial_extent, temporal_extent)

In [ ]:
collection = pystac.Collection(id=collection_id,
                     title=collection_title,
                     description=collection_description,
                     extent=collection_extent)

In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-7.57216793459,
     49.959999905,
     1.68153079591,
     58.6350001085]]},
  'temporal': {'interval': [['2011-12-16T00:00:00Z', None]]}},
 'license': 'proprietary'}

In [ ]:
collection.providers = [
    pystac.Provider(name='Airbus Defence & Space', roles=['producer'], url='https://www.airbus.com/space.html'),
    pystac.Provider(name='UK Space Agency', roles=['licensor'], url='https://www.gov.uk/government/organisations/uk-space-agency'),
    pystac.Provider(name='Satellite Applications Catapult', roles=['processor'], url='https://sa.catapult.org.uk/'),
    pystac.Provider(name='Satellite Applications Catapult', roles=['host'], url='https://sa.catapult.org.uk/')
]

In [ ]:
collection.add_item(item)

In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-7.57216793459,
     49.959999905,
     1.68153079591,
     58.6350001085]]},
  'temporal': {'interval': [['2011-12-16T00:00:00Z', None]]}},
 'license': 'proprietary',
 'providers': [{'name': 'Airbus Defence & Space',
   'roles': ['producer'],
   'url': 'https://www.airbus.com/space.html'},
  {'name': 'UK Space Agency',
   'roles': ['licensor'],
   'url': 'https://www.gov.uk/government/organisations/uk-space-agency'},
  {'name': 'Satellite Applications Catapult',
   'roles': ['processor'],
   'url': 'https://sa.catapult.org.uk/'},
  {'name': '

In [ ]:
collection.update_extent_from_items()
collection.extent.to_dict()

{'spatial': {'bbox': [[-1.903199885312827,
    51.62869184186537,
    -1.592242985175717,
    51.70382196570706]]},
 'temporal': {'interval': [['2018-09-29T11:10:00Z', '2018-09-29T11:10:00Z']]}}

In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-1.903199885312827,
     51.62869184186537,
     -1.592242985175717,
     51.70382196570706]]},
  'temporal': {'interval': [['2018-09-29T11:10:00Z',
     '2018-09-29T11:10:00Z']]}},
 'license': 'proprietary',
 'providers': [{'name': 'Airbus Defence & Space',
   'roles': ['producer'],
   'url': 'https://www.airbus.com/space.html'},
  {'name': 'UK Space Agency',
   'roles': ['licensor'],
   'url': 'https://www.gov.uk/government/organisations/uk-space-agency'},
  {'name': 'Satellite Applications Catapult',
   'roles': ['processor'],
   'url': 'ht

In [ ]:
from pathlib import Path

In [ ]:
root_path = str(Path.home() / collection.id)

In [ ]:
root_path

'/root/uksa-ssgp-pleiades'

In [ ]:
collection.normalize_hrefs(root_path)

<Collection id=uksa-ssgp-pleiades>

In [ ]:
collection.validate_all()

In [ ]:
collection.save(pystac.CatalogType.SELF_CONTAINED)

In [ ]:
collection.describe()

* <Collection id=uksa-ssgp-pleiades>
  * <Item id=Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101>


In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root',
   'href': './collection.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101/Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201809291110008_FR1_PX_W002N51_0417_007113614104101.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': '/root/uksa-ssgp-pleiades/collection.json',
   'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-1.903199885312827,
     51.62869184186537,
     -1.592242985175717,
     51.70382196570706]]},
  'temporal': {'interval': [['2018-09-29T11:10:00Z',
     '2018-09-29T11:10:00Z']]}},
 'license': 'proprietary',
 'providers': 

In [ ]:
import os
from http.server import HTTPServer, SimpleHTTPRequestHandler

os.chdir(root_path)

class CORSRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Cache-Control', 'no-store, no-cache, must-revalidate')
        return super(CORSRequestHandler, self).end_headers()


with HTTPServer(('localhost', 5555), CORSRequestHandler) as httpd:
    httpd.serve_forever()

KeyboardInterrupt: 

## Export

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_rediswq.ipynb.
Converted 00_utils.ipynb.
Converted 01A_pleiades.ipynb.
Converted 01B_pleiades_prep_worker.ipynb.
Converted 02A_spot.ipynb.
Converted index.ipynb.
